In [1]:
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .appName("DeltaLakeDemo") \
    .getOrCreate()

# Create sample DataFrame
df = spark.createDataFrame([
    (1, "Alice"),
    (2, "Bob"),
    (3, "Cathy")
], ["id", "name"])

# Write as Delta table
df.write.format("delta").mode("overwrite").save("/data/people")

# Read Delta table
df2 = spark.read.format("delta").load("/data/people")
df2.show()


+---+-----+
| id| name|
+---+-----+
|  3|Cathy|
|  1|Alice|
|  2|  Bob|
+---+-----+



# 🔁 1. ACID Transactions

✅ **What Are ACID Transactions?**  
ACID stands for:

- **Atomicity** → each write is all-or-nothing  
- **Consistency** → data moves from one valid state to another  
- **Isolation** → concurrent writes don’t interfere  
- **Durability** → once a write is committed, it's permanent  

Delta Lake brings ACID to data lakes by using a transaction log — just like a database does — even though the data is stored in flat files (Parquet) in object storage or local disk.

---

🔧 **How Delta Achieves ACID Transactions**  
Let’s break it down into mechanisms.

### 1. Transaction Log (`_delta_log/`)
Every write operation (insert, delete, update, merge, etc.) creates a new log file like:

```logs
/data/people-delta/_delta_log/00000000000000000000.json
/data/people-delta/_delta_log/00000000000000000001.json
```


Each log file contains:
- **Metadata** (`metaData`)  
- **Operation info** (`commitInfo`)  
- **A list of files added or removed** (`add`, `remove` entries)  

🧠 The data itself is immutable Parquet files. Delta just manages which files are “active” by listing them in the latest log version.

---

### 2. Atomicity
When you write data:

- Delta writes the new Parquet files first.  
- Only if *all* new files are successfully written, it appends a JSON log file with `add` entries.  
- If anything fails, nothing gets committed to the `_delta_log`, and the table state remains unchanged.  

This is atomic: **either the whole change is visible, or none of it is.**

---

### 3. Isolation via File Locking
Delta uses concurrency control:

- **Local file systems:** it can use file locks  
- **Cloud stores like S3:** it uses a protocol called *Optimistic Concurrency Control (OCC)*  

With OCC:
- A transaction reads the current state (e.g., version 12)  
- It prepares new changes based on that state  
- When it commits, it checks whether version 12 is still the latest  
- If someone else has committed version 13 in the meantime → it fails and retries  

This keeps multiple concurrent writers isolated and avoids conflicts.

---

### 4. Durability
The `_delta_log` is an append-only log, and once written:

- It is **never modified**  
- It is considered the **source of truth**  
- Even if the cluster crashes, you can rebuild the table’s current state from these logs  

That’s your **durability guarantee**.

---

⚠️ **Does It Affect Performance?**  
Yes, but in a good way (mostly)!

✅ **Benefits**
- Querying is faster: because the log tells you exactly which files to read, instead of scanning the whole folder  
- Appends and updates are efficient and safe  

⚠️ **Trade-offs**
- Small writes result in many small Parquet files → can affect read performance  
- That’s why Delta has **Optimize** and **Z-Ordering** to compact and organize files  
- For large-scale use, write performance is *slightly* slower than blind dumping to Parquet — but you gain **huge advantages** in reliability, auditability, and maintainability.


In [6]:
import shutil
import json

# Path to your Delta table
path = "/data/test-acid"

# Delete the entire folder
shutil.rmtree(path, ignore_errors=True)

df = spark.createDataFrame([
    (1, "Alice"),
    (2, "Bob"),
    (3, "Cathy")
], ["id", "name"])

# Write some data
df.write.format("delta").mode("overwrite").save("/data/test-acid")

# Append new row
spark.createDataFrame([(4, "test")], ["id", "name"]) \
    .write.format("delta").mode("append").save("/data/test-acid")

from IPython.display import display, JSON

log_dir = "/data/test-acid/_delta_log"
log_files = sorted([f for f in os.listdir(log_dir) if f.endswith(".json")])

for file in log_files:
    print(f"\n>>> Log version: {file}")
    with open(os.path.join(log_dir, file)) as f:
        for line in f.readlines():
            parsed = json.loads(line)
            display(JSON(parsed))


>>> Log version: 00000000000000000000.json


<IPython.core.display.JSON object>

<IPython.core.display.JSON object>

<IPython.core.display.JSON object>

<IPython.core.display.JSON object>

<IPython.core.display.JSON object>

<IPython.core.display.JSON object>


>>> Log version: 00000000000000000001.json


<IPython.core.display.JSON object>

<IPython.core.display.JSON object>

## _delta_logs Breakdown

### 🧾 `commitInfo` Example Breakdown

This is the metadata for the commit.

- **`timestamp`**:  
  The time when the operation was committed (in milliseconds since Unix epoch).

- **`operation`**:  
  This log entry represents a `WRITE` operation (it could also be `INSERT`, `UPDATE`, `DELETE`, etc.).  
  Here, it’s a `WRITE` operation.

- **`operationParameters`**:
  - **`mode`**: `Overwrite` — the data is being written to the table, replacing the previous content.  
  - **`partitionBy`**: `[]` — indicates the partitioning scheme. An empty list means the table isn’t partitioned.  
  - **`isolationLevel`**: `Serializable` — the highest level of isolation (no other operations can interfere during this transaction).  
  - **`isBlindAppend`**: `False` — this is not a simple append; it’s a full overwrite.

- **`operationMetrics`**:  
  Provides stats for the operation:
  - **`numFiles`**: 4 new files were written.  
  - **`numOutputRows`**: 3 rows of data were written.  
  - **`numOutputBytes`**: 2522 bytes — the total size of the written data.

- **`txnId`**:  
  The transaction ID (unique to the transaction).  
  Used to track this specific operation in Delta’s transaction log.


### 📦 `metaData` Example Breakdown

Contains schema and metadata information for the Delta table.

- **`id`**:  
  A unique ID for the Delta table (used internally for reference).

- **`format`**:  
  The format used for storing data.  
  Here, it's `parquet`, the underlying storage format for Delta.

- **`schemaString`**:  
  The schema of the table, defined as a JSON string. This tells you the structure of the data (columns, types, etc.).  
  In this case:
  - `id`: A `long` field (nullable)  
  - `name`: A `string` field (nullable)

- **`partitionColumns`**:  
  The table isn’t partitioned (`[]`).

- **`configuration`**:  
  Additional configuration (empty here).

- **`createdTime`**:  
  The time the table was created (in milliseconds since Unix epoch).


### ➕ `add` Entry Breakdown

This entry indicates that a new Parquet file was added to the Delta table.

- **`path`**:  
  The path to the newly added Parquet file.

- **`partitionValues`**:  
  Empty here since the table isn’t partitioned.

- **`size`**:  
  The size of the Parquet file in bytes (`719` bytes).

- **`modificationTime`**:  
  The timestamp when the file was last modified.

- **`dataChange`**:  
  `True` means that this file contains **data changes** (not just metadata).

- **`stats`**:  
  The statistics for the data in this file:
  - **`numRecords`**: This file contains 1 record.  
  - **`minValues`** and **`maxValues`**:  
    The minimum and maximum values of the fields in the file.  
    Helps optimize queries (e.g., for pruning files based on min/max).  
  - **`nullCount`**:  
    The number of null values in each field.


# ⏳ Time Travel in Delta Lake

Time Travel in Delta Lake allows you to query historical versions of your data, even after updates, deletes, or appends. This feature is possible thanks to Delta Lake’s transaction log.

---

### 🔍 How Time Travel Works

Delta Lake uses the `_delta_log` to track all changes to the table. Each write operation (INSERT, UPDATE, DELETE, etc.) appends a new JSON file to the log, containing metadata about the operation.

#### 🗂 Transaction Log
Each commit is stored as a new JSON file in `_delta_log/`, including:
- The operation type (`WRITE`, `UPDATE`, `DELETE`)
- The schema at that point
- Data files written or removed
- The resulting table version

#### 🔢 Versions
Each commit increments the table version:
- Version 0 is the initial state
- Version 1 is the next, and so on
- Each commit includes a unique `txnId`

#### 🧾 Snapshotting
When querying, Delta reads the latest log version and constructs a snapshot.  
To read an older version, it uses the corresponding log version and rebuilds the table as it was.

---

### ⚙️ Mechanics Behind Time Travel

- **Immutable Logs**: Once written, logs are never changed.
- **Immutable Data**: Parquet files are also immutable.
- **Versioning**: Delta tracks which files belong to each version.
- **Garbage Collection**: Old versions are cleaned up via `VACUUM` after a retention period.

---

### 📅 How to Query Historical Versions

#### By Version Number
```python
df = spark.read.format("delta") \
    .option("versionAsOf", 0) \
    .load("path_to_table")
df.show()

#### 🕰️ By Timestamp
Delta finds the most recent version that was committed **before** the specified timestamp and returns the snapshot of that version.

---

### 📊 Performance Considerations of Time Travel

#### 🔄 **Read Performance**:
Time travel operations are efficient because Delta doesn’t re-read all data. Instead, it:
- Uses the transaction log and metadata to only read necessary files.
- Might need to read multiple files, but avoids scanning the entire table.
- Leverages Delta’s query optimizer, which efficiently prunes data based on the log's statistics.

#### 🗑️ **Garbage Collection**:
- Time travel doesn't indefinitely increase storage size.
- Delta performs garbage collection to remove old versions when they are no longer needed.
- This process is controlled by the retention period and can be manually triggered using the `VACUUM` command to delete old data files no longer referenced.

#### 💾 **Storage Overhead**:
- There is some overhead due to new Parquet files and logs being created with each version.
- However, this is minimized by Delta’s efficient handling of small incremental writes.

---

### 📝 Example Use Case

Suppose you want to go back to a version of the Delta table as it was on **2025-04-20 at 15:30**. Here’s how you would proceed:

1. **Check Table History** to find the closest version:
```python
delta_table = DeltaTable.forPath(spark, "path_to_table")
history_df = delta_table.history()
history_df.show(truncate=False)
```

In [9]:
from delta.tables import DeltaTable

delta_table = DeltaTable.forPath(spark, "/data/test-acid")
history_df = delta_table.history()
history_df.show(truncate=False)

+-------+-----------------------+------+--------+---------+--------------------------------------+----+--------+---------+-----------+--------------+-------------+-----------------------------------------------------------+------------+-----------------------------------+
|version|timestamp              |userId|userName|operation|operationParameters                   |job |notebook|clusterId|readVersion|isolationLevel|isBlindAppend|operationMetrics                                           |userMetadata|engineInfo                         |
+-------+-----------------------+------+--------+---------+--------------------------------------+----+--------+---------+-----------+--------------+-------------+-----------------------------------------------------------+------------+-----------------------------------+
|1      |2025-04-23 09:19:56.172|null  |null    |WRITE    |{mode -> Append, partitionBy -> []}   |null|null    |null     |0          |Serializable  |true         |{numFiles -> 2, nu

In [12]:
delta_table.delete("id = 1")

In [13]:
history_df = delta_table.history()
history_df.show(truncate=False)

+-------+-----------------------+------+--------+---------+--------------------------------------+----+--------+---------+-----------+--------------+-------------+---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+------------+-----------------------------------+
|version|timestamp              |userId|userName|operation|operationParameters                   |job |notebook|clusterId|readVersion|isolationLevel|isBlindAppend|operationMetrics                                                                                                                                                                                                           |userMetadata|engineInfo                         |
+-------+-----------------------+------+--------+---------+--------------------------------------+----+--------+---------+-----------+

In [14]:
spark.read.format("delta").option("versionAsOf", 1).load("/data/test-acid").filter("id = 1").show()

+---+-----+
| id| name|
+---+-----+
|  1|Alice|
|  1| test|
+---+-----+

+---+----+
| id|name|
+---+----+
+---+----+



In [15]:
spark.read.format("delta").load("/data/test-acid").filter("id = 1").show()

+---+----+
| id|name|
+---+----+
+---+----+



How Deletes Are Handled in Delta Lake
Delta Lake is append-only under the hood. It never modifies existing data files directly. Instead, it follows a copy-on-write strategy:

1. Query Optimization and File Pruning
When you run a DELETE or an UPDATE, Delta first identifies which Parquet files contain the rows to be deleted using the statistics in the _delta_log (like minValues, maxValues, and nullCounts).

Only those files are read and rewritten. Others are untouched.

2. Rewrite with Changes
Delta reads the affected Parquet files, applies the delete condition (e.g., WHERE id = 42), and writes new files with the remaining (non-deleted) records.

The old Parquet files are logically removed — they are not physically deleted immediately but are marked as removed in the _delta_log.

3. Transaction Log Update
The _delta_log records:

A new version with a commitInfo block describing the DELETE operation.

A set of remove actions, each pointing to a deleted (now obsolete) data file.

A set of add actions for the newly written files.

So from a transactional point of view:

The table instantly reflects the updated state (with the records deleted).

Old files still exist and can be used to reconstruct previous versions — which is what enables time travel.

In [16]:
from IPython.display import display, JSON

log_dir = "/data/test-acid/_delta_log"
log_files = sorted([f for f in os.listdir(log_dir) if f.endswith(".json")])

for file in log_files:
    print(f"\n>>> Log version: {file}")
    with open(os.path.join(log_dir, file)) as f:
        for line in f.readlines():
            parsed = json.loads(line)
            display(JSON(parsed))


>>> Log version: 00000000000000000000.json


<IPython.core.display.JSON object>

<IPython.core.display.JSON object>

<IPython.core.display.JSON object>

<IPython.core.display.JSON object>

...

<IPython.core.display.JSON object>


>>> Log version: 00000000000000000002.json


<IPython.core.display.JSON object>

<IPython.core.display.JSON object>

<IPython.core.display.JSON object>

🕰️ Why Time Travel Leads to Storage Growth
Every Delta operation (write, update, delete) doesn't modify files in-place — it writes new Parquet files and logs a new version in _delta_log.

So over time:

More versions accumulate in _delta_log/

More old Parquet files (not referenced by the latest table state) accumulate

This can lead to:

Higher storage costs

Slightly slower read performance, especially for metadata-heavy operations like full scans or schema inference, if you don’t manage it

🔍 How This Affects Performance
Reading the latest snapshot is fast, because Delta uses a transaction log and checkpoints to resolve it quickly.

But:

The more logs there are, the longer it takes to process them during startup (if no recent checkpoint exists).

If you use versionAsOf or timestampAsOf, Delta might have to scan logs backwards and reconstruct the state from previous versions.

Over time, many small files can also lead to file listing overhead, especially on cloud storage.

🧹 How to Manage This (Best Practices)
Here’s how Delta Lake helps you keep things clean and fast:

✅ 1. VACUUM – Physical Cleanup
Delta retains old files for a default of 7 days, even if they’re not used by the current table version. This is to support time travel.

You can manually or automatically remove obsolete files:

python
Copy
Edit
from delta.tables import DeltaTable
delta_table = DeltaTable.forPath(spark, "path_to_table")
delta_table.vacuum(retentionHours=168)  # 7 days = 168 hours
Tip: You can set a shorter retention (e.g. 1 hour) only if you're sure you don’t need time travel to those states:

python
Copy
Edit
delta_table.vacuum(retentionHours=1)  # Dangerous if others rely on old versions
✅ 2. CHECKPOINTING – Faster Metadata Resolution
Delta Lake creates checkpoint files every N versions (default: 10). These are compact versions of the state up to that point. That way, Delta can rebuild the current snapshot quickly without replaying the entire log history.

You can force a checkpoint manually:

python
Copy
Edit
delta_table = DeltaTable.forPath(spark, "path_to_table")
spark._jsparkSession.sessionState().catalog().asTableCatalog().loadTableMetadata(
    spark._jsparkSession.sessionState().sqlParser().parseTableIdentifier("delta.`path_to_table`")
)
(That’s the internal way — but usually you just let Delta manage it.)

✅ 3. LOG CLEANUP – Transaction Log Compaction
Delta also retains all those .json log files (one per version). These are small, but over time many can add up.

If you're using a cloud storage bucket, having 100k log files can slow down file listing operations. Delta handles this by:

Using checkpoints

Compaction of logs

You can also manually remove old logs if you’ve vacuumed and don’t need time travel to those versions anymore.

✅ 4. Tuning Retention Policies
You can configure these defaults globally or per table:

python
Copy
Edit
spark.conf.set("spark.databricks.delta.retentionDurationCheck.enabled", "false")  # Needed for <7d
Or set table properties:

sql
Copy
Edit
ALTER TABLE delta.`path_to_table` SET TBLPROPERTIES ('delta.deletedFileRetentionDuratio

In [27]:
from pyspark.sql import SparkSession

builder = SparkSession.builder \
    .appName("DeltaCheckpointVacuumDemo") \
    .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension") \
    .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog") \
    .config("spark.sql.shuffle.partitions", "1")  # to reduce files for testing

spark = builder.getOrCreate()
spark.conf.set("spark.databricks.delta.retentionDurationCheck.enabled", "false")

In [28]:
import shutil
from pathlib import Path

base_path = "/data/delta/checkpoint_test"
shutil.rmtree(base_path, ignore_errors=True)  # Clean slate

df = spark.range(1).withColumnRenamed("id", "val")
df.write.format("delta").mode("overwrite").save(base_path)

In [29]:
#Step 3: Inspect _delta_log/
import os

log_files = sorted(os.listdir(f"{base_path}/"))
for f in log_files:
    print(f)

.part-00000-270402e2-5b13-440f-b7b8-39475c1e2dc9-c000.snappy.parquet.crc
.part-00011-a9937862-201c-4282-b1cc-db8efa759d4d-c000.snappy.parquet.crc
_delta_log
part-00000-270402e2-5b13-440f-b7b8-39475c1e2dc9-c000.snappy.parquet
part-00011-a9937862-201c-4282-b1cc-db8efa759d4d-c000.snappy.parquet


In [30]:
#This will generate 11 commit files and automatically trigger a checkpoint at version 10.
from delta.tables import DeltaTable

dt = DeltaTable.forPath(spark, base_path)

for i in range(1, 12):
    df = spark.range(i, i + 1).withColumnRenamed("id", "val")
    df.write.format("delta").mode("append").save(base_path)
    print(f"Appended version {i}")

Appended version 1
Appended version 2
Appended version 3
Appended version 4
Appended version 5
Appended version 6
Appended version 7
Appended version 8
Appended version 9
Appended version 10
Appended version 11


In [31]:
#Step 3: Inspect parquet files
import os

log_files = sorted(os.listdir(f"{base_path}/"))
for f in log_files:
    print(f)

.part-00000-11da9a11-3f34-4247-a787-89fac4f509cd-c000.snappy.parquet.crc
.part-00000-270402e2-5b13-440f-b7b8-39475c1e2dc9-c000.snappy.parquet.crc
.part-00000-4b81e43d-fd73-4740-95ea-0afe6901dabe-c000.snappy.parquet.crc
.part-00000-6d68a7ee-59b9-4e62-8c6b-d8c51ba2d979-c000.snappy.parquet.crc
.part-00000-8ce5a978-e36d-42af-8757-9ba38e0ad4be-c000.snappy.parquet.crc
.part-00000-9758411a-2580-4b2b-8ccc-1fe73d4a7fe7-c000.snappy.parquet.crc
.part-00000-caa341f1-b64e-48b3-95d2-ea09a13de127-c000.snappy.parquet.crc
.part-00000-d3814381-4e98-4e82-b4c6-e72b84eb2f06-c000.snappy.parquet.crc
.part-00000-ece421c8-b395-41be-b4ca-84b9ddbd9217-c000.snappy.parquet.crc
.part-00000-f36285f7-de4b-4278-b8b7-096475dfa988-c000.snappy.parquet.crc
.part-00000-f86c7aac-016a-487d-a3ef-0a67dbf29566-c000.snappy.parquet.crc
.part-00000-ff4b16d4-7ee8-4bcc-a07e-00b5f5942423-c000.snappy.parquet.crc
.part-00011-09c33597-309a-4a9d-b242-cc54eab5a897-c000.snappy.parquet.crc
.part-00011-211c2702-6c60-4d7f-8c64-536818841fa3-c0

In [32]:
#Step 3: Inspect _delta_log/
import os

log_files = sorted(os.listdir(f"{base_path}/_delta_log"))
for f in log_files:
    print(f)

.00000000000000000000.json.crc
.00000000000000000001.json.crc
.00000000000000000002.json.crc
.00000000000000000003.json.crc
.00000000000000000004.json.crc
.00000000000000000005.json.crc
.00000000000000000006.json.crc
.00000000000000000007.json.crc
.00000000000000000008.json.crc
.00000000000000000009.json.crc
.00000000000000000010.checkpoint.parquet.crc
.00000000000000000010.json.crc
.00000000000000000011.json.crc
._last_checkpoint.crc
00000000000000000000.json
00000000000000000001.json
00000000000000000002.json
00000000000000000003.json
00000000000000000004.json
00000000000000000005.json
00000000000000000006.json
00000000000000000007.json
00000000000000000008.json
00000000000000000009.json
00000000000000000010.checkpoint.parquet
00000000000000000010.json
00000000000000000011.json
_last_checkpoint


➡️ You should now see:

00000000000000000000.json through 00000000000000000010.json

A file like 00000000000000000010.checkpoint.parquet
This is the checkpoint file: a compacted representation of the table state up to version 10.

In [36]:
dt.history().show()

+-------+--------------------+------+--------+------------+--------------------+----+--------+---------+-----------+-----------------+-------------+--------------------+------------+--------------------+
|version|           timestamp|userId|userName|   operation| operationParameters| job|notebook|clusterId|readVersion|   isolationLevel|isBlindAppend|    operationMetrics|userMetadata|          engineInfo|
+-------+--------------------+------+--------+------------+--------------------+----+--------+---------+-----------+-----------------+-------------+--------------------+------------+--------------------+
|     13|2025-04-23 13:39:...|  null|    null|  VACUUM END|{status -> COMPLE...|null|    null|     null|         12|SnapshotIsolation|         true|{numDeletedFiles ...|        null|Apache-Spark/3.3....|
|     12|2025-04-23 13:39:...|  null|    null|VACUUM START|{retentionCheckEn...|null|    null|     null|         11|SnapshotIsolation|         true|{numFilesToDelete...|        null|Ap

In [34]:
#🧹 Step 4: Run VACUUM
dt.vacuum(retentionHours=0)  # Only for testing — don't do in production!

DataFrame[]

The deltaTable.history() function reads from the JSON commit files in _delta_log/, which are not deleted by VACUUM. So you can still trace all the changes made to the table
While the history is intact, the ability to time travel to older versions may be affected depending on what VACUUM removed.

In [37]:
dt.history().show()

+-------+--------------------+------+--------+------------+--------------------+----+--------+---------+-----------+-----------------+-------------+--------------------+------------+--------------------+
|version|           timestamp|userId|userName|   operation| operationParameters| job|notebook|clusterId|readVersion|   isolationLevel|isBlindAppend|    operationMetrics|userMetadata|          engineInfo|
+-------+--------------------+------+--------+------------+--------------------+----+--------+---------+-----------+-----------------+-------------+--------------------+------------+--------------------+
|     13|2025-04-23 13:39:...|  null|    null|  VACUUM END|{status -> COMPLE...|null|    null|     null|         12|SnapshotIsolation|         true|{numDeletedFiles ...|        null|Apache-Spark/3.3....|
|     12|2025-04-23 13:39:...|  null|    null|VACUUM START|{retentionCheckEn...|null|    null|     null|         11|SnapshotIsolation|         true|{numFilesToDelete...|        null|Ap

You'll likely still see:

JSON logs

Checkpoint file

But old Parquet data files (no longer referenced) will be gone.

In [24]:
#Step 3: Inspect parquet files
import os

log_files = sorted(os.listdir(f"{base_path}/"))
for f in log_files:
    print(f)

.part-00011-04669adb-8cfc-405c-83de-0f2511b61806-c000.snappy.parquet.crc
.part-00011-149528b6-5498-4835-bc1b-361df3dc445a-c000.snappy.parquet.crc
.part-00011-3b068554-c092-42b1-8836-2c7edfc0e0a1-c000.snappy.parquet.crc
.part-00011-48f0ee4e-2497-4e68-a5a2-ae4d252d309e-c000.snappy.parquet.crc
.part-00011-4d41da4f-fef5-4979-83ee-d13fe85f930e-c000.snappy.parquet.crc
.part-00011-54702229-d38e-436c-a2ab-4c7ea650f713-c000.snappy.parquet.crc
.part-00011-630df688-8db7-4995-9734-b989b4c3ab7e-c000.snappy.parquet.crc
.part-00011-643eacbb-92e3-432d-bb22-141b6dd3968a-c000.snappy.parquet.crc
.part-00011-a114ee84-5f01-4781-8e25-dbea719e3ee0-c000.snappy.parquet.crc
.part-00011-cbb048bb-06a4-46b6-b556-92359ee16c0f-c000.snappy.parquet.crc
.part-00011-d2d8df03-371e-44e0-b8f2-c858831204a7-c000.snappy.parquet.crc
.part-00011-e70089c8-bf8c-478d-81f1-ae6b50d9d18e-c000.snappy.parquet.crc
_delta_log
part-00011-04669adb-8cfc-405c-83de-0f2511b61806-c000.snappy.parquet
part-00011-149528b6-5498-4835-bc1b-361df3dc44

In [16]:
#Then re-inspect the _delta_log/ folder:
log_files_after_vacuum = sorted(os.listdir(f"{base_path}/_delta_log"))
for f in log_files_after_vacuum:
    print(f)

.00000000000000000000.json.crc
.00000000000000000001.json.crc
.00000000000000000002.json.crc
.00000000000000000003.json.crc
.00000000000000000004.json.crc
.00000000000000000005.json.crc
.00000000000000000006.json.crc
.00000000000000000007.json.crc
.00000000000000000008.json.crc
.00000000000000000009.json.crc
.00000000000000000010.checkpoint.parquet.crc
.00000000000000000010.json.crc
.00000000000000000011.json.crc
.00000000000000000012.json.crc
.00000000000000000013.json.crc
.00000000000000000014.json.crc
.00000000000000000015.json.crc
._last_checkpoint.crc
00000000000000000000.json
00000000000000000001.json
00000000000000000002.json
00000000000000000003.json
00000000000000000004.json
00000000000000000005.json
00000000000000000006.json
00000000000000000007.json
00000000000000000008.json
00000000000000000009.json
00000000000000000010.checkpoint.parquet
00000000000000000010.json
00000000000000000011.json
00000000000000000012.json
00000000000000000013.json
00000000000000000014.json
0000000

In [26]:
from IPython.display import display, JSON
import json

log_dir = f"{base_path}/_delta_log"
log_files = sorted([f for f in os.listdir(log_dir) if f.endswith(".json")])

for file in log_files:
    print(f"\n>>> Log version: {file}")
    with open(os.path.join(log_dir, file)) as f:
        for line in f.readlines():
            parsed = json.loads(line)
            display(JSON(parsed))


>>> Log version: 00000000000000000000.json


<IPython.core.display.JSON object>

<IPython.core.display.JSON object>

<IPython.core.display.JSON object>

<IPython.core.display.JSON object>

...

<IPython.core.display.JSON object>


>>> Log version: 00000000000000000012.json


<IPython.core.display.JSON object>


>>> Log version: 00000000000000000013.json


<IPython.core.display.JSON object>

🛡 3. Schema Enforcement
Try writing data with a wrong schema:

In [43]:
right_schema = spark.createDataFrame([
    (99999, "John")
], ["id", "name"])

# This will fail!
right_schema.write.format("delta").mode("append").save("/data/schema-people-delta")

In [44]:
wrong_schema = spark.createDataFrame([
    (99999, "John", "wrong_column_here")
], ["id", "name", "oops"])

# This will fail!
wrong_schema.write.format("delta").mode("append").save("/data/schema-people-delta")

AnalysisException: A schema mismatch detected when writing to the Delta table (Table ID: 950f499f-9acb-4364-b628-7449e65640b1).
To enable schema migration using DataFrameWriter or DataStreamWriter, please set:
'.option("mergeSchema", "true")'.
For other operations, set the session configuration
spark.databricks.delta.schema.autoMerge.enabled to "true". See the documentation
specific to the operation for details.

Table schema:
root
-- id: long (nullable = true)
-- name: string (nullable = true)


Data schema:
root
-- id: long (nullable = true)
-- name: string (nullable = true)
-- oops: string (nullable = true)

         

🔄 4. Schema Evolution
But we can allow evolution like this:

In [57]:
evolving = spark.createDataFrame([
    (10001, "New Guy", 22, "Fantasyland", "guy@delta.com", "extra info")
], ["id", "name", "age", "country", "email", "note"])

evolving.write.format("delta").mode("append") \
    .option("mergeSchema", "true") \
    .save("/data/schema-people-delta")

In [58]:
spark.read.format("delta").load("/data/schema-people-delta").printSchema()

root
 |-- id: long (nullable = true)
 |-- name: string (nullable = true)
 |-- age: long (nullable = true)
 |-- country: string (nullable = true)
 |-- email: string (nullable = true)
 |-- note: string (nullable = true)



In [59]:
spark.read.format("delta").load("/data/schema-people-delta").show()

+-----+----------+----+-----------+-----------------+----------+
|   id|      name| age|    country|            email|      note|
+-----+----------+----+-----------+-----------------+----------+
|10001|   New Guy|  22|Fantasyland|    guy@delta.com|extra info|
|99999|   New Guy|  22|Fantasyland|    guy@delta.com|extra info|
|10001|Alice Test|  36|  Neverland|alice@updated.com|          |
|10002|  New User|  40|    Wakanda|    new@delta.com|          |
|99999|      John|null|       null|             null|      null|
+-----+----------+----+-----------+-----------------+----------+



🧬 5. Upserts (MERGE INTO)
Let’s simulate an upsert (update existing or insert new):

In [60]:
dt.toDF().printSchema()

root
 |-- id: long (nullable = true)
 |-- name: string (nullable = true)
 |-- age: long (nullable = true)
 |-- country: string (nullable = true)
 |-- email: string (nullable = true)
 |-- note: string (nullable = true)



In [61]:
dt = DeltaTable.forPath(spark, "/data/schema-people-delta")
updates = spark.createDataFrame([
    (10001, "Alice Test", 36, "Neverland", "alice@updated.com",""),  # updated
    (10002, "New User", 40, "Wakanda", "new@delta.com","")           # inserted
], ["id", "name", "age", "country", "email","note"])

dt.alias("target").merge(
    updates.alias("source"),
    "target.id = source.id"
).whenMatchedUpdateAll().whenNotMatchedInsertAll().execute()

In [62]:
dt.toDF().show()

+-----+----------+----+-----------+-----------------+----------+
|   id|      name| age|    country|            email|      note|
+-----+----------+----+-----------+-----------------+----------+
|10001|Alice Test|  36|  Neverland|alice@updated.com|          |
|10001|Alice Test|  36|  Neverland|alice@updated.com|          |
|10002|  New User|  40|    Wakanda|    new@delta.com|          |
|99999|   New Guy|  22|Fantasyland|    guy@delta.com|extra info|
|99999|      John|null|       null|             null|      null|
+-----+----------+----+-----------+-----------------+----------+



🔥 6. Delete & Update
Delta supports SQL-like updates and deletes:

In [63]:
# Delete people older than 75
dt.delete("age > 35")

# Update country name
dt.update("country = 'Neverland'", {"country": "'Fantasy Realm'"})

In [64]:
dt.toDF().show()

+-----+-------+----+-----------+-------------+----------+
|   id|   name| age|    country|        email|      note|
+-----+-------+----+-----------+-------------+----------+
|99999|New Guy|  22|Fantasyland|guy@delta.com|extra info|
|99999|   John|null|       null|         null|      null|
+-----+-------+----+-----------+-------------+----------+



📦 7. File Compaction (Optimize)
Delta creates many small parquet files. You can compact them:

In [66]:
#Step 3: Inspect parquet files
import os

log_files = sorted(os.listdir(f"/data/schema-people-delta"))
for f in log_files:
    print(f)

.part-00000-2590be05-8f9a-4def-b16c-fc3a3ceb07fa-c000.snappy.parquet.crc
.part-00000-94e50324-6549-441a-8071-ab3f0e5a2206-c000.snappy.parquet.crc
.part-00000-b054e596-bf15-40a1-b781-c57c2259e45a-c000.snappy.parquet.crc
.part-00000-b7a2b6e5-457c-4e04-ad8f-3c62716c44e1-c000.snappy.parquet.crc
.part-00000-d69dc4c4-ee68-4bda-875c-fa6ea055c6a0-c000.snappy.parquet.crc
.part-00000-f5dbb791-68ea-42bb-b2ef-ca39e93e88ca-c000.snappy.parquet.crc
.part-00011-360c616d-c8c1-40cd-94cd-a825448dbd34-c000.snappy.parquet.crc
.part-00011-4c5462e2-a77b-4fbc-819e-8fd446f25b5b-c000.snappy.parquet.crc
.part-00011-7f789d97-502f-47f0-b469-a7807898c4f1-c000.snappy.parquet.crc
_delta_log
part-00000-2590be05-8f9a-4def-b16c-fc3a3ceb07fa-c000.snappy.parquet
part-00000-94e50324-6549-441a-8071-ab3f0e5a2206-c000.snappy.parquet
part-00000-b054e596-bf15-40a1-b781-c57c2259e45a-c000.snappy.parquet
part-00000-b7a2b6e5-457c-4e04-ad8f-3c62716c44e1-c000.snappy.parquet
part-00000-d69dc4c4-ee68-4bda-875c-fa6ea055c6a0-c000.snappy.

In [72]:
DeltaTable.forPath(spark, "/data/schema-people-delta").optimize().executeCompaction()  # If you're using Delta OSS with support

DataFrame[path: string, metrics: struct<numFilesAdded:bigint,numFilesRemoved:bigint,filesAdded:struct<min:bigint,max:bigint,avg:double,totalFiles:bigint,totalSize:bigint>,filesRemoved:struct<min:bigint,max:bigint,avg:double,totalFiles:bigint,totalSize:bigint>,partitionsOptimized:bigint,zOrderStats:struct<strategyName:string,inputCubeFiles:struct<num:bigint,size:bigint>,inputOtherFiles:struct<num:bigint,size:bigint>,inputNumCubes:bigint,mergedFiles:struct<num:bigint,size:bigint>,numOutputCubes:bigint,mergedNumCubes:bigint>,numBatches:bigint,totalConsideredFiles:bigint,totalFilesSkipped:bigint,preserveInsertionOrder:boolean,numFilesSkippedToReduceWriteAmplification:bigint,numBytesSkippedToReduceWriteAmplification:bigint,startTimeMs:bigint,endTimeMs:bigint,totalClusterParallelism:bigint,totalScheduledTasks:bigint,autoCompactParallelismStats:struct<maxClusterActiveParallelism:bigint,minClusterActiveParallelism:bigint,maxSessionActiveParallelism:bigint,minSessionActiveParallelism:bigint>,de

In [73]:
import os

path = "/data/schema-people-delta"
log_files = sorted(os.listdir(path))

for f in log_files:
    full_path = os.path.join(path, f)
    if os.path.isfile(full_path):
        size_bytes = os.path.getsize(full_path)
        size_kb = round(size_bytes / 1024, 2)
        print(f"{f} - {size_kb} KB")

.part-00000-2590be05-8f9a-4def-b16c-fc3a3ceb07fa-c000.snappy.parquet.crc - 0.02 KB
.part-00000-75808793-61b2-4ea7-b9dc-214c22c54823-c000.snappy.parquet.crc - 0.02 KB
.part-00000-94e50324-6549-441a-8071-ab3f0e5a2206-c000.snappy.parquet.crc - 0.02 KB
.part-00000-b054e596-bf15-40a1-b781-c57c2259e45a-c000.snappy.parquet.crc - 0.02 KB
.part-00000-b7a2b6e5-457c-4e04-ad8f-3c62716c44e1-c000.snappy.parquet.crc - 0.02 KB
.part-00000-d69dc4c4-ee68-4bda-875c-fa6ea055c6a0-c000.snappy.parquet.crc - 0.01 KB
.part-00000-f5dbb791-68ea-42bb-b2ef-ca39e93e88ca-c000.snappy.parquet.crc - 0.02 KB
.part-00011-360c616d-c8c1-40cd-94cd-a825448dbd34-c000.snappy.parquet.crc - 0.02 KB
.part-00011-4c5462e2-a77b-4fbc-819e-8fd446f25b5b-c000.snappy.parquet.crc - 0.02 KB
.part-00011-7f789d97-502f-47f0-b469-a7807898c4f1-c000.snappy.parquet.crc - 0.02 KB
part-00000-2590be05-8f9a-4def-b16c-fc3a3ceb07fa-c000.snappy.parquet - 1.76 KB
part-00000-75808793-61b2-4ea7-b9dc-214c22c54823-c000.snappy.parquet - 1.82 KB
part-00000-94e